# Fletcher Project: Tweets Summarizing

In [1]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.corpus import treebank_chunk
from textblob import TextBlob
import pymongo
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
from operator import itemgetter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from __future__ import division, unicode_literals
import operator
import requests
from requests_oauthlib import OAuth1
from pymongo import MongoClient
import tweepy
import cnfg
import numpy as np
import re

In [580]:
client = MongoClient()

In [581]:
datatweets = client.dsbc.datatweets

In [582]:
datatweets.remove( { } )

{u'n': 200, u'ok': 1}

In [583]:
datatweets = client.dsbc.datatweets
for k in tweets:
    datatweets_d = {'text': k}
    datatweets.insert(datatweets_d)

In [585]:
tweets = []
cursor = datatweets.find({})
for d in range(datatweets.count()):
    tweet_text = cursor.next()
    print tweet_text
    tweets.append(tweet_text['text'])
tweets

{u'text': u'Nebraska abolishes death penalty http://t.co/lbxTTgw1Q9', u'_id': ObjectId('55677c76bb9ab402cb0849d6')}
{u'text': u'Is there a link between a bad economy and tooth decay? http://t.co/VU5TfBmnhG', u'_id': ObjectId('55677c76bb9ab402cb0849e4')}
{u'text': u'@katiebrenzel It is upsetting indeed.', u'_id': ObjectId('55677c76bb9ab402cb0849e6')}
{u'text': u'Treasury market in selloff mode as stocks recover http://t.co/hD5jjN0cro', u'_id': ObjectId('55677c76bb9ab402cb0849f8')}
{u'text': u'How meal delivery can make Whole Foods look cheap http://t.co/vlHCbzDoIy', u'_id': ObjectId('55677c76bb9ab402cb084a01')}
{u'text': u"McDonald's to stop reporting monthly sales figures http://t.co/f67yJqkHEk", u'_id': ObjectId('55677c76bb9ab402cb084a03')}
{u'text': u'RT @m_cof: The median salary for a stockbroker is a little over $71,000 a year http://t.co/buZHaglkxs http://t.co/911mZqbTKD', u'_id': ObjectId('55677c76bb9ab402cb084a36')}
{u'text': u'A man survived a shotgun blast to his chest thanks 

[u'Nebraska abolishes death penalty http://t.co/lbxTTgw1Q9',
 u'Is there a link between a bad economy and tooth decay? http://t.co/VU5TfBmnhG',
 u'@katiebrenzel It is upsetting indeed.',
 u'Treasury market in selloff mode as stocks recover http://t.co/hD5jjN0cro',
 u'How meal delivery can make Whole Foods look cheap http://t.co/vlHCbzDoIy',
 u"McDonald's to stop reporting monthly sales figures http://t.co/f67yJqkHEk",
 u'RT @m_cof: The median salary for a stockbroker is a little over $71,000 a year http://t.co/buZHaglkxs http://t.co/911mZqbTKD',
 u'A man survived a shotgun blast to his chest thanks to his iPhone: http://t.co/vZxN4cNnxd http://t.co/pTAvPJfS74',
 u'RT @jsadinolfi: The dollar CRUSHED IT today http://t.co/iSDl4Vza10 http://t.co/TdJ7LF27Kb',
 u'IRS says thieves stole tax information from 100,000 Americans http://t.co/KHgre4DzCZ',
 u"Facebook is just one of billionaire John Griffin's top tech picks. Check out the rest: http://t.co/l41c2eeVzI http://t.co/BzicpDpu8a",
 u'U.S. 

## Getting Twitter Data

In [557]:
config = cnfg.load(".twitter_config")

In [558]:
auth = tweepy.OAuthHandler(config['consumer_key'],
              config['consumer_secret'])
auth.set_access_token(config['access_token'],
              config['access_token_secret'])

In [559]:
api = tweepy.API(auth)

In [560]:
account = 'MarketWatch'
data_tweets = api.user_timeline(account, count=200)

## Setting chosen keyword

In [561]:
keyword ='china'

In [563]:
tweets = []
tweets_text = []
for tweet in data_tweets:
    tweets.append(tweet.text)
tweets

[u'Did you ever have a Zune? \n\nThe 12 worst American product flops: http://t.co/VYASsCaEfB http://t.co/72s6CHrOcW',
 u'RT @wmwitkowski: The Squeeze Is On: 42% of $GME outstanding shares shorted &amp; shares are up 7% after hours.http://t.co/6gHU7bwGpU http://t.c\u2026',
 u'U.S. stocks end slightly lower, on pace for weekly drop http://t.co/wHPWIKWB0p',
 u'RT @MKTWeconomics: Layoffs are rare, jobless claims data suggest\nhttp://t.co/gGi9K89X3g http://t.co/i7xbLph34I',
 u'One cop offers his tip for getting out of a speeding ticket: http://t.co/ZbECF6EtPI http://t.co/ttAiC2amYl',
 u'Dealerships are \u2018quietly supporting\u2019 Tesla\u2019s Elon Musk http://t.co/cvCpe58xtA http://t.co/oyWov2crLm',
 u"RT @m_cof: 'Neverland' lists for $100 million. Michael Jackson paid $19 million for the ranch in 1987 http://t.co/DCd69ErqZF http://t.co/4r\u2026",
 u'The most popular stores in America: http://t.co/LHruYKDmhM http://t.co/lSbPbVvBLa',
 u"Fed's Kocherlakota: Policy aim should be three more 

In [586]:
len(tweets)

200

In [565]:
stopw = [str('http'), str('https'), str('['), str(']'), str('rt'), str('%'), str('@'), str('#')]
stopngram = stopwords.words('english')
stopngram += ['.', ',', '(', ')', "'", '"', 'http', 'RT']

In [566]:
adjectives = {}
nouns = {}
adverbs = {}
verbs = {}
for t in tweets:
    words = word_tokenize(t)
    for p in pos_tag(words):
        tags = p[1][:2]
        keys = p[0].encode('utf8')
        keys = keys.lower()   
        if keys not in stopw and str('/') not in keys and "\\" not in r"%r" % keys:
            if tags == 'NN':
                nouns.setdefault(keys,0)
                nouns[keys] += 1

In [567]:
for words, count in sorted(nouns.items(), key = itemgetter(1), reverse=True)[:10]:
    print '%20s %i' % (words, count)

              stocks 23
                u.s. 20
         kitchennews 16
              dollar 13
                fifa 12
              nikkei 10
                asia 9
               kospi 8
                 asx 8
             college 7


In [568]:
#tweets_t = zip(*tweets)[0]
#tweets_r = zip(*tweets)[1]

In [569]:
stop = stopwords.words('english')

counter = defaultdict(int)

n = 2
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

    KitchenNews Asia 8
          U.S stocks 7
          Upper East 4
           East Side 4
         Whole Foods 4
   community college 3
         money tight 3
              0.1 HK 3
        Michael Kors 3
     drone companies 3


In [570]:
stop = stopwords.words('english')

counter = defaultdict(int)

n = 3
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

     Upper East Side 4
community college students 3
  Fitting Upper East 3
    another way spot 2
   Asia early Nikkei 2
 jobless claims data 2
 expand market power 2
KitchenNews Asia midday 2
  highest nine years 2
    When money tight 2


In [571]:
keyword = str(keyword)
keyword = keyword.lower()
keyword

'china'

In [572]:
tweets_match = []
for r in tweets:
    if keyword in r.lower():
        tweets_match.append(r)

len(tweets_match)

5

In [573]:
adjectives = {}
nouns = {}
adverbs = {}
verbs = {}
for t in tweets_match:
    words = word_tokenize(t)
    for p in pos_tag(words):
        tags = p[1][:2]
        keys = p[0].encode('utf8')
        keys = keys.lower()   
        if keys not in stopw and str('/') not in keys and keyword not in keys and "\\" not in r"%r" % keys:
            if tags == 'JJ':
                adjectives.setdefault(keys,0)
                adjectives[keys] += 1
            if tags == 'NN':
                nouns.setdefault(keys,0)
                nouns[keys] += 1
            if tags == 'RB':
                adverbs.setdefault(keys,0)
                adverbs[keys] += 1
            if tags == 'VB':
                verbs.setdefault(keys,0)
                verbs[keys] += 1

In [574]:
for words, count in sorted(verbs.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

              expand 2
             outlook 1
              plunge 1
                 mad 1
                sets 1


In [575]:
for words, count in sorted(adjectives.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

               giant 2
                rush 1


In [576]:
for words, count in sorted(nouns.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

                fund 3
               power 2
                gold 2
                cuts 2
          lauraliuhe 2


In [577]:
for words, count in sorted(adverbs.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

                most 1


In [578]:
stop = stopwords.words('english')

counter = defaultdict(int)

n = 2
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

        market power 2
          China sets 2
       expand market 2
          giant gold 2
           gold fund 2


In [579]:
stop = stopwords.words('english')

counter = defaultdict(int)

n = 3
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

    gold fund expand 2
     sets giant gold 2
     giant gold fund 2
 expand market power 2
    China sets giant 2
